### Load up selenium and the Chrome webdriver

In [4]:
import selenium
from selenium import webdriver
from selenium.common import NoSuchElementException, ElementNotInteractableException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

ModuleNotFoundError: No module named 'selenium'

In [ ]:
driver = webdriver.Chrome()

In [ ]:
driver.get('https://generalssb-prod.ec.njit.edu/BannerExtensibility/customPage/page/stuRegCrseSched')

### In the left navigation, find the 16th entry ("CS") and click on it

#### This will cause the main content area to load all of the CS courses

In [ ]:
driver.implicitly_wait(5)
left_link = driver.find_element(By.ID, 'pbid-subjListTableSubjectLink-16')

In [ ]:
cs_link = left_link.find_element(by=By.TAG_NAME, value='a')
cs_link.click()

In [ ]:
my_span = driver.find_element(by=By.ID, value='pbid-courseListSectionDetailSections-0')

In [ ]:
print(my_span)

In [ ]:
course_headers = my_span.find_elements(by=By.TAG_NAME, value="H4")

In [ ]:
course_tables = my_span.find_elements(by=By.TAG_NAME, value="table")

In [ ]:
print("I have {} headers and {} tables".format(len(course_headers), len(course_tables)))

In [ ]:
def create_column_headings(table_headers):
    loc_columns = ['Course_name']
    for i in range(len(table_headers)):
        loc_columns.append(table_headers[i].text)
    return loc_columns

In [ ]:
def create_course_listings(c_headers, c_tables):
    c_list = []
    for i in range(len(c_headers)):
        course_name = c_headers[i].text
#       collect the rows for each section of the course
        t_rows = c_tables[i].find_elements(by=By.CLASS_NAME, value='success')
#       for each row, get all of the cells
        for k in range(len(t_rows)):
            course_row = []
            course_row.append(course_name)
            t_cells = t_rows[k].find_elements(by=By.TAG_NAME, value='td')
            for l in range(len(t_cells)):
                course_row.append(t_cells[l].text)
            c_list.append(course_row)
    return c_list


In [ ]:
# create column headings by using the 'th' cells of the first table
t_headers = course_tables[0].find_elements(by=By.TAG_NAME, value="th")
columns = create_column_headings(t_headers)

In [ ]:
# use the list of course_headers and the list of course_tables to build
#   individual rows - one for each course/section
course_list = create_course_listings(course_headers, course_tables)
df = pd.DataFrame(course_list, columns=columns)
driver.quit()

In [ ]:
df

In [ ]:
df.describe()

In [ ]:
df['Max'] = df['Max'].astype(int)
df['Now'] = df['Now'].astype(int)

In [ ]:
df['Open'] = df['Max'] - df['Now']

In [ ]:
### potential rows to delete
        # skip Independent Study, Master's Project, Master's Thesis 
#        if course_name.startswith("CS 488") or course_name.startswith("CS 700B") \
#        or course_name.startswith("CS 701B") or course_name.startswith("CS 725") \
#        or course_name.startswith("CS 792") or course_name.startswith("CS 726") \
#        or course_name.startswith("CS 790A"):

In [ ]:
df = df[~df['Course_name'].str.contains('CS 488')]
df = df[~df['Course_name'].str.contains('CS 700B')]
df = df[~df['Course_name'].str.contains('CS 701B')]
df = df[~df['Course_name'].str.contains('CS 725')]
df = df[~df['Course_name'].str.contains('CS 792')]
df = df[~df['Course_name'].str.contains('CS 790A')]
df = df[~df['Course_name'].str.contains('CS 726')]

df

In [ ]:
df[['Instructor', 'Max', 'Now']][200:250]

In [ ]:
df[['Instructor','Open']].groupby(['Instructor']).sum().tail(50)


In [ ]:
df['Course_name'].unique()